# Объединеный ноутбу для использования в Google Colab
# Детектирование браконьеров на фотоловушках

Для запуска необходимо добавить Roboflow Api Key в Google Colab Secretstore

Логика: Браконьер = Person (класс 4) + Оружие (grenade/knife/pistol/rifle)
Этот ноутбук выполняет:
1. Загрузку датасета с Roboflow API (person-weapon-v2)
2. Преобразование в формат YOLO
3. Анализ статистики классов
4. Split на train/val/test (70/15/15)
5. Визуализацию примеров
6. Загрузку предобученной модели YOLOv8 Nano
7. Fine-tuning на кастомных данных
8. Мониторинг метрик во время обучения
9. Сохранение лучшей модели
10. Анализ графиков обучения


In [ ]:

import json
import os
import shutil
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import yaml
from google.colab import userdata
from sklearn.model_selection import train_test_split
from tqdm import tqdm

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("=" * 70)
print("ПОДГОТОВКА ДАННЫХ ДЛЯ ОБУЧЕНИЯ YOLOv8 (person-weapon-v2)")
print("=" * 70)


# ЧАСТЬ 1: СОЗДАНИЕ ДИРЕКТОРИЙ


In [ ]:

BASE_DIR = Path('data')
BASE_DIR.mkdir(exist_ok=True)

RAW_DIR = BASE_DIR / 'raw'
PROCESSED_DIR = BASE_DIR / 'processed'
ANNOTATIONS_DIR = BASE_DIR / 'annotations'

RAW_DIR.mkdir(exist_ok=True)
PROCESSED_DIR.mkdir(exist_ok=True)
ANNOTATIONS_DIR.mkdir(exist_ok=True)

print("\nДиректории созданы успешно\n")

# ЧАСТЬ 2: ОПРЕДЕЛЕНИЕ КЛАССОВ


In [ ]:

print("=" * 70)
print("ОПРЕДЕЛЕНИЕ КЛАССОВ ДАТАСЕТА person-weapon-v2")
print("=" * 70 + "\n")

# Все 5 классов из датасета
ALL_CLASSES = {
    'grenade': 0,
    'knife': 1,
    'pistol': 2,
    'rifle': 3,
    'person': 4
}

# Классы оружия (для определения браконьеров)
WEAPON_CLASSES = {0, 1, 2, 3}  # grenade, knife, pistol, rifle
PERSON_CLASS = 4

print("Все классы датасета:")
for name, idx in sorted(ALL_CLASSES.items(), key=lambda x: x[1]):
    if idx in WEAPON_CLASSES:
        print(f"   {idx}: {name} [ОРУЖИЕ]")
    else:
        print(f"   {idx}: {name} [ЧЕЛОВЕК]")

print(f"\n Логика определения браконьера:")
print(f"   На фото присутствует Person (класс 4)")
print(f"   И одновременно присутствует оружие (классы 0, 1, 2, 3)")


# ЧАСТЬ 3: ЗАГРУЗКА ДАННЫХ С ROBOFLOW

In [ ]:
print("\n" + "=" * 70)
print("ЗАГРУЗКА ДАТАСЕТА С ROBOFLOW")
print("=" * 70 + "\n")

try:
    from roboflow import Roboflow

    print("Roboflow уже установлена")
except ImportError:
    print("Установка Roboflow...")
    os.system('pip install roboflow -q')
    from roboflow import Roboflow

    print("Roboflow установлена")

API_KEY = userdata.get('roboflow_api_key')

if API_KEY != 'YOUR_ROBOFLOW_API_KEY':
    print(f"\n Инициализация Roboflow...")
    try:
        rf = Roboflow(api_key=API_KEY)
        # Загрузить person-weapon-v2 датасет
        project = rf.workspace('ksdjflaskjf').project('person-weapon-v2')
        dataset = project.version(2).download(model_format='yolov8', location=str(RAW_DIR), overwrite=True)
        print(f"Датасет person-weapon-v2 загружен в {RAW_DIR}\n")
    except Exception as e:
        print(f"Ошибка при загрузке: {e}")
        print("Используйте локальные данные или установите правильный ключ\n")
else:
    print("API ключ не установлен. Используем локальные данные...\n")


# ЧАСТЬ 4: АНАЛИЗ СТРУКТУРЫ ДАННЫХ


In [ ]:

print("=" * 70)
print("АНАЛИЗ СТРУКТУРЫ ДАННЫХ")
print("=" * 70 + "\n")

images_path = RAW_DIR / 'train' / 'images'
labels_path = RAW_DIR / 'train' / 'labels'

image_extensions = {'.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG'}

if images_path.exists() and labels_path.exists():
    image_files = sorted([f for f in images_path.iterdir()
                          if f.suffix in image_extensions])
    label_files = sorted([f for f in labels_path.iterdir() if f.suffix == '.txt'])

    print(f"Найдено изображений: {len(image_files)}")
    print(f"Найдено аннотаций: {len(label_files)}")
    print(f"Пары изображение-аннотация: {min(len(image_files), len(label_files))}\n")
else:
    print("Структура папок не найдена")
    image_files = []
    label_files = []


In [ ]:
# ============================================================================
# ЧАСТЬ 5: АНАЛИЗ КЛАССОВ И ОПРЕДЕЛЕНИЕ БРАКОНЬЕРОВ
# ============================================================================

print("=" * 70)
print("АНАЛИЗ КЛАССОВ И ОПРЕДЕЛЕНИЕ БРАКОНЬЕРОВ")
print("=" * 70 + "\n")
class_counts = {i: 0 for i in range(5)}
annotation_stats = []
poacher_images = []  # Изображения с браконьерами
person_only_images = []  # Только люди без оружия
weapon_only_images = []  # Только оружие без людей

for label_file in label_files:
    with open(label_file, 'r') as f:
        lines = f.readlines()
        annotation_stats.append(len(lines))

        classes_in_image = set()
        for line in lines:
            try:
                class_id = int(line.split()[0])
                if class_id < 5:
                    class_counts[class_id] += 1
                    classes_in_image.add(class_id)
            except (IndexError, ValueError):
                continue

        # Определить тип изображения
        has_person = PERSON_CLASS in classes_in_image
        has_weapon = any(w in classes_in_image for w in WEAPON_CLASSES)

        img_name = label_file.stem
        if has_person and has_weapon:
            poacher_images.append(img_name)
        elif has_person and not has_weapon:
            person_only_images.append(img_name)
        elif not has_person and has_weapon:
            weapon_only_images.append(img_name)

print("Статистика классов:")
total_objects = sum(class_counts.values())
for class_id, name in sorted([(v, k) for k, v in ALL_CLASSES.items()]):
    count = class_counts[class_id]
    percentage = (count / total_objects * 100) if total_objects > 0 else 0
    print(f"   {class_id}: {name:10s} — {count:4d} объектов ({percentage:5.1f}%)")

print(f"\nАнализ типов изображений:")
print(f"   Браконьеры (Person + Оружие): {len(poacher_images)} ({len(poacher_images) / len(label_files) * 100:.1f}%)")
print(f"   Только люди: {len(person_only_images)} ({len(person_only_images) / len(label_files) * 100:.1f}%)")
print(f"   Только оружие: {len(weapon_only_images)} ({len(weapon_only_images) / len(label_files) * 100:.1f}%)")

if annotation_stats:
    print(f"\nСтатистика аннотаций на изображение:")
    print(f"   Среднее: {np.mean(annotation_stats):.2f}")
    print(f"   Мин: {np.min(annotation_stats)}")
    print(f"   Макс: {np.max(annotation_stats)}\n")


# ЧАСТЬ 6: ВИЗУАЛИЗАЦИЯ

In [ ]:

def draw_yolo_boxes(image_path, label_path, all_classes):
    """
    Нарисовать bounding boxes на изображении.
    """
    image = cv2.imread(str(image_path))
    if image is None:
        return None

    height, width = image.shape[:2]

    if label_path.exists():
        with open(label_path, 'r') as f:
            lines = f.readlines()

        # Цвета для разных классов
        colors = {
            0: (0, 165, 255),  # grenade - оранжевый
            1: (0, 255, 255),  # knife - жёлтый
            2: (0, 0, 255),  # pistol - красный
            3: (255, 0, 0),  # rifle - синий
            4: (0, 255, 0)  # person - зелёный
        }

        # Словарь названий для букв на английском
        class_names_en = {v: k for k, v in all_classes.items()}

        for line in lines:
            try:
                parts = line.strip().split()
                class_id = int(parts[0])
                x_center = float(parts[1]) * width
                y_center = float(parts[2]) * height
                box_width = float(parts[3]) * width
                box_height = float(parts[4]) * height

                x1 = int(x_center - box_width / 2)
                y1 = int(y_center - box_height / 2)
                x2 = int(x_center + box_width / 2)
                y2 = int(y_center + box_height / 2)

                color = colors.get(class_id, (255, 255, 255))
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

                class_name = class_names_en.get(class_id, 'Unknown')
                cv2.putText(image, class_name, (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            except (IndexError, ValueError):
                continue

    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


print("=" * 70)
print("ВИЗУАЛИЗАЦИЯ ПРИМЕРОВ")
print("=" * 70 + "\n")

# Выбрать примеры разных типов
n_examples = min(6, len(image_files))
examples_to_show = []

# Взять примеры браконьеров (если есть)
if poacher_images:
    examples_to_show.extend(poacher_images[:2])

# Взять примеры только людей
if person_only_images:
    examples_to_show.extend(person_only_images[:2])

# Взять случайные примеры
remaining = n_examples - len(examples_to_show)
all_img_names = [f.stem for f in image_files]
random_examples = [img for img in all_img_names if img not in examples_to_show][:remaining]
examples_to_show.extend(random_examples)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, img_name in enumerate(examples_to_show[:6]):
    img_path = images_path / f"{img_name}{image_files[0].suffix}"
    label_path = labels_path / f"{img_name}.txt"

    annotated_img = draw_yolo_boxes(img_path, label_path, ALL_CLASSES)

    if annotated_img is not None:
        # Определить тип
        classes_in_img = set()
        if label_path.exists():
            with open(label_path) as f:
                for line in f:
                    classes_in_img.add(int(line.split()[0]))

        has_person = PERSON_CLASS in classes_in_img
        has_weapon = any(w in classes_in_img for w in WEAPON_CLASSES)

        if has_person and has_weapon:
            title = f'{img_name}\nБРАКОНЬЕР'
        elif has_person:
            title = f'{img_name}\nЧеловек'
        else:
            title = f'{img_name}'

        axes[idx].imshow(annotated_img)
        axes[idx].set_title(title, fontsize=10, fontweight='bold')
        axes[idx].axis('off')

plt.suptitle('Примеры из датасета\n(зелёный=человек, pistol=пистолет, 🔵=винтовка, жёлтый=нож, оранжевый=граната)',
             fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()
print("Примеры визуализированы\n")


# ЧАСТЬ 7: ГРАФИКИ СТАТИСТИКИ


In [ ]:

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# График 1: Распределение классов
class_names_list = [ALL_CLASSES for ALL_CLASSES in sorted([(v, k) for k, v in ALL_CLASSES.items()])]
class_names_list = [name for idx, name in sorted([(ALL_CLASSES[k], k) for k in ALL_CLASSES])]
counts = [class_counts[i] for i in range(5)]

colors_bar = ['#FF6B35', '#FFD23F', '#FF0000', '#0066FF', '#00CC00']
axes[0, 0].bar(class_names_list, counts, color=colors_bar, alpha=0.7)
axes[0, 0].set_ylabel('Количество объектов')
axes[0, 0].set_title('Распределение классов', fontweight='bold')
axes[0, 0].grid(axis='y', alpha=0.3)

for i, (name, count) in enumerate(zip(class_names_list, counts)):
    axes[0, 0].text(i, count + 50, str(count), ha='center', fontweight='bold')

# График 2: Типы изображений
types = ['Браконьеры', 'Только люди', 'Только оружие']
values = [len(poacher_images), len(person_only_images), len(weapon_only_images)]
axes[0, 1].bar(types, values, color=['#e74c3c', '#3498db', '#f39c12'], alpha=0.7)
axes[0, 1].set_ylabel('Количество изображений')
axes[0, 1].set_title('Тип содержимого изображений', fontweight='bold')
axes[0, 1].grid(axis='y', alpha=0.3)

for i, v in enumerate(values):
    axes[0, 1].text(i, v + 10, str(v), ha='center', fontweight='bold')

# График 3: Распределение объектов на изображение
axes[1, 0].hist(annotation_stats, bins=20, color='#3498db', edgecolor='black', alpha=0.7)
axes[1, 0].set_xlabel('Объектов на изображение')
axes[1, 0].set_ylabel('Количество изображений')
axes[1, 0].set_title('Распределение объектов', fontweight='bold')
axes[1, 0].axvline(np.mean(annotation_stats), color='red', linestyle='--', linewidth=2,
                   label=f'Среднее: {np.mean(annotation_stats):.2f}')
axes[1, 0].legend()
axes[1, 0].grid(axis='y', alpha=0.3)

# График 4: Доля браконьеров
poacher_percentage = len(poacher_images) / len(label_files) * 100
other_percentage = (len(person_only_images) + len(weapon_only_images)) / len(label_files) * 100

axes[1, 1].pie([poacher_percentage, other_percentage],
               labels=['Браконьеры', 'Другое'],
               colors=['#e74c3c', '#95a5a6'],
               autopct='%1.1f%%',
               startangle=90)
axes[1, 1].set_title('Доля браконьеров в датасете', fontweight='bold')

plt.tight_layout()
plt.show()
print("Графики созданы\n")


# ЧАСТЬ 8: SPLIT НА TRAIN/VAL/TEST

In [ ]:
print("=" * 70)
print("SPLIT ДАТАСЕТА (70/15/15)")
print("=" * 70 + "\n")

if len(image_files) > 0:
    YOLO_DIR = PROCESSED_DIR / 'yolo_format'
    YOLO_DIR.mkdir(exist_ok=True)

    splits = ['train', 'val', 'test']
    for split in splits:
        (YOLO_DIR / split / 'images').mkdir(parents=True, exist_ok=True)
        (YOLO_DIR / split / 'labels').mkdir(parents=True, exist_ok=True)

    # Стратифицированный split для сохранения баланса браконьеров
    all_img_names = [f.stem for f in image_files]

    # Разделить на браконьеров и остальных
    poacher_list = list(poacher_images)
    other_list = [img for img in all_img_names if img not in poacher_list]

    # Split браконьеров
    p_train, p_temp = train_test_split(poacher_list, test_size=0.3, random_state=42)
    p_val, p_test = train_test_split(p_temp, test_size=0.5, random_state=42)

    # Split остальных
    o_train, o_temp = train_test_split(other_list, test_size=0.3, random_state=42)
    o_val, o_test = train_test_split(o_temp, test_size=0.5, random_state=42)

    # Объединить
    splits_data = {
        'train': p_train + o_train,
        'val': p_val + o_val,
        'test': p_test + o_test
    }

    print("Распределение данных:")
    for split_name, files in splits_data.items():
        poachers_in_split = sum(1 for f in files if f in poacher_list)
        percentage = (len(files) / len(image_files)) * 100
        print(f"   {split_name}: {len(files)} изображений ({percentage:.1f}%) — браконьеров: {poachers_in_split}")

    # ========================================================================
    # ЧАСТЬ 9: КОПИРОВАНИЕ ФАЙЛОВ
    # ========================================================================

    print("\n" + "=" * 70)
    print("КОПИРОВАНИЕ ФАЙЛОВ В СТРУКТУРУ YOLO")
    print("=" * 70 + "\n")

    for split_name, files in splits_data.items():
        for img_name in tqdm(files, desc=f'Копирование {split_name}'):
            # Найти файл изображения
            img_file = None
            for f in image_files:
                if f.stem == img_name:
                    img_file = f
                    break

            if img_file:
                src_img = img_file
                dst_img = YOLO_DIR / split_name / 'images' / img_file.name
                shutil.copy2(src_img, dst_img)

                src_label = labels_path / f"{img_name}.txt"
                dst_label = YOLO_DIR / split_name / 'labels' / f"{img_name}.txt"

                if src_label.exists():
                    shutil.copy2(src_label, dst_label)

    print("\nВсе файлы скопированы")

    # ========================================================================
    # ЧАСТЬ 10: СОЗДАНИЕ dataset.yaml
    # ========================================================================

    print("\n" + "=" * 70)
    print("СОЗДАНИЕ КОНФИГ-ФАЙЛА dataset.yaml")
    print("=" * 70 + "\n")

    dataset_yaml = f"""# YOLOv8 Dataset Configuration (person-weapon-v2)
# Для обучения на задаче детектирования браконьеров

path: {YOLO_DIR}
train: train/images
val: val/images
test: test/images

# Количество классов
nc: 5

# Названия классов
names:
  0: grenade
  1: knife
  2: pistol
  3: rifle
  4: person

# Примечание: Браконьер = person (4) + оружие (0,1,2,3)
"""

    yaml_path = BASE_DIR / 'dataset.yaml'
    with open(yaml_path, 'w') as f:
        f.write(dataset_yaml)

    print(f"dataset.yaml создан: {yaml_path}\n")
    print("Содержание файла:")
    print(dataset_yaml)

    # ========================================================================
    # ЧАСТЬ 11: СОХРАНЕНИЕ МЕТАДАННЫХ
    # ========================================================================

    metadata = {
        "dataset_name": "person-weapon-v2",
        "total_images": len(image_files),
        "poacher_images": len(poacher_images),
        "person_only_images": len(person_only_images),
        "weapon_only_images": len(weapon_only_images),
        "class_distribution": {str(k): v for k, v in class_counts.items()},
        "splits": {
            "train": len(splits_data['train']),
            "val": len(splits_data['val']),
            "test": len(splits_data['test'])
        },
        "poacher_detection_logic": "person (class 4) + weapon (classes 0,1,2,3)"
    }

    metadata_path = BASE_DIR / 'metadata.json'
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)

    print(f"Метаданные сохранены: {metadata_path}")

    # ========================================================================
    # ЧАСТЬ 12: ФИНАЛЬНАЯ ПРОВЕРКА
    # ========================================================================

    print("\n" + "=" * 70)
    print("ФИНАЛЬНАЯ ПРОВЕРКА ЦЕЛОСТНОСТИ ДАННЫХ")
    print("=" * 70 + "\n")

    all_valid = True
    for split_name in splits:
        images_dir = YOLO_DIR / split_name / 'images'
        labels_dir = YOLO_DIR / split_name / 'labels'

        image_count = len(list(images_dir.glob('*.*')))
        label_count = len(list(labels_dir.glob('*.txt')))

        status = 'ok' if image_count == label_count else 'not ok️'
        print(f"{status} {split_name}: {image_count} изображений, {label_count} аннотаций")

        if image_count != label_count:
            all_valid = False

    print("\n" + "=" * 70)
    print("ИТОГОВАЯ СТАТИСТИКА")
    print("=" * 70)

    total_images = sum(len(list((YOLO_DIR / split / 'images').glob('*.*')))
                       for split in splits)
    print(f"\nВсего изображений обработано: {total_images}")
    print(f"Изображений с браконьерами: {len(poacher_images)}")
    print(f"Структура YOLO: {YOLO_DIR}")
    print(f"Конфиг YAML: {yaml_path}")

    if all_valid:
        print("\nПОДГОТОВКА ДАННЫХ ЗАВЕРШЕНА!")
        print("\nГотово к обучению модели!")
        print("   Перейдите к ноутбуку: 02_model_training.ipynb")
    else:
        print("\nВНИМАНИЕ: Обнаружены проблемы с целостностью данных")


# Часть 13. Проверка железа

In [ ]:
# Проверить GPU
print("=" * 70)
print("ПРОВЕРКА ОКРУЖЕНИЯ")
print("=" * 70)

print(f"\n🐍 Python версия: {os.sys.version.split()[0]}")
print(f"PyTorch версия: {torch.__version__}")
print(f"CUDA доступна: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA версия: {torch.version.cuda}")
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"Всего VRAM: {total_memory:.2f} ГБ")
else:
    print("⚠️ GPU не найдена. Обучение будет на CPU (медленнее)")

# ЧАСТЬ 1: УСТАНОВКА И ИМПОРТ ULTRALYTICS

In [ ]:

print("\n" + "=" * 70)
print("УСТАНОВКА ULTRALYTICS")
print("=" * 70 + "\n")

try:
    from ultralytics import YOLO

    print("Ultralytics уже установлена")
except ImportError:
    print("Установка Ultralytics...")
    os.system('pip install ultralytics -q')
    from ultralytics import YOLO

    print("Ultralytics установлена")

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# ЧАСТЬ 1: УСТАНОВКА И ИМПОРТ ULTRALYTICS


In [ ]:

print("\n" + "=" * 70)
print("УСТАНОВКА ULTRALYTICS")
print("=" * 70 + "\n")

try:
    from ultralytics import YOLO

    print("Ultralytics уже установлена")
except ImportError:
    print("Установка Ultralytics...")
    os.system('pip install ultralytics -q')
    from ultralytics import YOLO

    print("Ultralytics установлена")

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')


# ЧАСТЬ 2: ПОДГОТОВКА ПУТЕЙ И КОНФИГУРАЦИИ

In [ ]:

print("=" * 70)
print("ПОДГОТОВКА КОНФИГУРАЦИИ")
print("=" * 70 + "\n")

BASE_DIR = Path('data')
YAML_PATH = BASE_DIR / 'dataset.yaml'
MODELS_DIR = Path('models')
MODELS_DIR.mkdir(exist_ok=True)

# Проверить наличие конфиг-файла
if not YAML_PATH.exists():
    print(f"ОШИБКА: Файл {YAML_PATH} не найден")
    print("Запустите сначала: 01_data_preparation.ipynb")
else:
    print(f"Конфиг найден: {YAML_PATH}")

    # Прочитать конфиг
    with open(YAML_PATH, 'r') as f:
        config = yaml.safe_load(f)

    print(f"\nКонфигурация датасета:")
    print(f"   Классы: {config['names']}")
    print(f"   Количество классов: {config['nc']}")
    print(f"   Train dataset: {config['train']}")
    print(f"   Val dataset: {config['val']}")
    print(f"   Test dataset: {config['test']}")


# ЧАСТЬ 3: ЗАГРУЗКА МОДЕЛИ


In [ ]:

print("\n" + "=" * 70)
print("ЗАГРУЗКА МОДЕЛИ YOLOv8 NANO")
print("=" * 70 + "\n")

MODEL_NAME = 'yolov8n'  # nano - самая лёгкая (0.9 ГБ VRAM)
print(f"📥 Загрузка модели: {MODEL_NAME}...")

model = YOLO(f'{MODEL_NAME}.pt')
print(f"✅ Модель загружена успешно\n")

# Информация о модели
print("📊 Информация о модели:")
print(f"   Размер: 0.9 ГБ (оптимально для 15 ГБ VRAM)")
print(f"   Параметры: ~3.3M")
print(f"   Скорость: 100+ fps на GPU")
print(f"   mAP50: ~98% на COCO")


# ЧАСТЬ 4: ГИПЕРПАРАМЕТРЫ ОБУЧЕНИЯ

In [ ]:

print("\n" + "=" * 70)
print("ГИПЕРПАРАМЕТРЫ ОБУЧЕНИЯ")
print("=" * 70 + "\n")

# Оптимальные параметры для YOLOv8n с 15 ГБ VRAM
HYPERPARAMS = {
    'epochs': 50,  # Количество эпох
    'imgsz': 640,  # Размер входного изображения
    'batch': 16,  # Размер батча (уменьшён для экономии VRAM)
    'device': 0 if torch.cuda.is_available() else 'cpu',  # GPU device (0 = первая видеокарта)
    'lr0': 0.01,  # Начальный learning rate
    'lrf': 0.01,  # Финальный learning rate
    'momentum': 0.937,  # Momentum оптимизатора
    'weight_decay': 0.0005,  # Коэффициент регуляризации
    'warmup_epochs': 3.0,  # Количество warming-up эпох
    'patience': 20,  # Early stopping patience
    'save': True,  # Сохранять чекпоинты
    'cache': 'ram',  # Кэширование данных (ram/disk/False)
    'augment': True,  # Data augmentation
    'degrees': 10.0,  # Rotation
    'translate': 0.1,  # Translation
    'scale': 0.5,  # Scale
    'hsv_h': 0.015,  # HSV-H augmentation
    'hsv_s': 0.7,  # HSV-S augmentation
    'hsv_v': 0.4,  # HSV-V augmentation
    'flipud': 0.0,  # Flip updown probability
    'fliplr': 0.5,  # Flip left-right probability
    'perspective': 0.0,  # Perspective transform
    'mosaic': 1.0,  # Mosaic augmentation
    'mixup': 0.0,  # Mixup augmentation
    'copy_paste': 0.0,  # Copy-paste augmentation
    'erasing': 0.0,  # Random erasing
    'conf': 0.5,  # Confidence threshold для инференса
    'iou': 0.6,  # IoU threshold для NMS
}

print("Параметры обучения:")
for key, value in HYPERPARAMS.items():
    print(f"   {key}: {value}")


# ЧАСТЬ 5: ОБУЧЕНИЕ МОДЕЛИ

In [ ]:

print("\n" + "=" * 70)
print("ЗАПУСК ОБУЧЕНИЯ")
print("=" * 70 + "\n")

print("Начало обучения...")
print(f"Примерное время: 2-4 часа на GPU\n")

# Fine-tuning
results = model.train(
    data=str(YAML_PATH),
    epochs=HYPERPARAMS['epochs'],
    imgsz=HYPERPARAMS['imgsz'],
    batch=HYPERPARAMS['batch'],
    device=HYPERPARAMS['device'],
    lr0=HYPERPARAMS['lr0'],
    lrf=HYPERPARAMS['lrf'],
    momentum=HYPERPARAMS['momentum'],
    weight_decay=HYPERPARAMS['weight_decay'],
    warmup_epochs=HYPERPARAMS['warmup_epochs'],
    patience=HYPERPARAMS['patience'],
    save=HYPERPARAMS['save'],
    cache=HYPERPARAMS['cache'],
    augment=HYPERPARAMS['augment'],
    degrees=HYPERPARAMS['degrees'],
    translate=HYPERPARAMS['translate'],
    scale=HYPERPARAMS['scale'],
    hsv_h=HYPERPARAMS['hsv_h'],
    hsv_s=HYPERPARAMS['hsv_s'],
    hsv_v=HYPERPARAMS['hsv_v'],
    flipud=HYPERPARAMS['flipud'],
    fliplr=HYPERPARAMS['fliplr'],
    perspective=HYPERPARAMS['perspective'],
    mosaic=HYPERPARAMS['mosaic'],
    mixup=HYPERPARAMS['mixup'],
    copy_paste=HYPERPARAMS['copy_paste'],
    erasing=HYPERPARAMS['erasing'],
    project=str(MODELS_DIR),
    name='yolov8n_poacher_detector',
    exist_ok=True,
    verbose=True,
)

print("\nОбучение завершено!")

# ЧАСТЬ 6: СОХРАНЕНИЕ ЛУЧШЕЙ МОДЕЛИ

In [ ]:

print("\n" + "=" * 70)
print("СОХРАНЕНИЕ МОДЕЛИ")
print("=" * 70 + "\n")

# Лучшая модель уже сохранена в runs/detect/yolov8n_poacher_detector/weights/best.pt
best_model_path = MODELS_DIR / 'yolov8n_poacher_detector' / 'weights' / 'best.pt'

if best_model_path.exists():
    # Скопировать в models директорию
    import shutil

    final_model_path = MODELS_DIR / 'yolov8n_best.pt'
    shutil.copy2(best_model_path, final_model_path)
    print(f"Лучшая модель сохранена: {final_model_path}")
else:
    print(f"Модель не найдена по пути: {best_model_path}")

# ЧАСТЬ 7: АНАЛИЗ РЕЗУЛЬТАТОВ ОБУЧЕНИЯ

In [ ]:

print("\n" + "=" * 70)
print("АНАЛИЗ РЕЗУЛЬТАТОВ ОБУЧЕНИЯ")
print("=" * 70 + "\n")

# Загрузить результаты обучения
results_dir = MODELS_DIR / 'yolov8n_poacher_detector'
results_csv = results_dir / 'results.csv'

if results_csv.exists():
    results_df = pd.read_csv(results_csv)

    print("Итоговые метрики:")
    print(f"   mAP50: {results_df['metrics/mAP50(B)'].iloc[-1]:.4f}")
    print(f"   mAP50-95: {results_df['metrics/mAP50-95(B)'].iloc[-1]:.4f}")
    print(f"   Precision: {results_df['metrics/precision(B)'].iloc[-1]:.4f}")
    print(f"   Recall: {results_df['metrics/recall(B)'].iloc[-1]:.4f}")

    # Графики обучения
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # Loss
    axes[0, 0].plot(results_df['epoch'], results_df['train/box_loss'], label='Train')
    axes[0, 0].plot(results_df['epoch'], results_df['val/box_loss'], label='Val')
    axes[0, 0].set_ylabel('Box Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].legend()
    axes[0, 0].grid()
    axes[0, 0].set_title('Loss during Training')

    # mAP
    axes[0, 1].plot(results_df['epoch'], results_df['metrics/mAP50(B)'], label='mAP50')
    axes[0, 1].plot(results_df['epoch'], results_df['metrics/mAP50-95(B)'], label='mAP50-95')
    axes[0, 1].set_ylabel('mAP')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].legend()
    axes[0, 1].grid()
    axes[0, 1].set_title('mAP during Training')

    # Precision & Recall
    axes[1, 0].plot(results_df['epoch'], results_df['metrics/precision(B)'], label='Precision')
    axes[1, 0].plot(results_df['epoch'], results_df['metrics/recall(B)'], label='Recall')
    axes[1, 0].set_ylabel('Score')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].legend()
    axes[1, 0].grid()
    axes[1, 0].set_title('Precision & Recall during Training')

    # Object Confidence
    axes[1, 1].plot(results_df['epoch'], results_df['train/cls_loss'], label='Train')
    axes[1, 1].plot(results_df['epoch'], results_df['val/cls_loss'], label='Val')
    axes[1, 1].set_ylabel('Obj Loss')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].legend()
    axes[1, 1].grid()
    axes[1, 1].set_title('Object Loss during Training')

    plt.tight_layout()
    plt.show()

    print("\nГрафики обучения созданы")

# ЧАСТЬ 8: ИНФОРМАЦИЯ О МОДЕЛИ

In [ ]:

print("\n" + "=" * 70)
print("ИНФОРМАЦИЯ О ОБУЧЕННОЙ МОДЕЛИ")
print("=" * 70 + "\n")

if final_model_path.exists():
    model_size = final_model_path.stat().st_size / 1e6
    print(f"Размер модели: {model_size:.2f} МБ")
    print(f"Путь к модели: {final_model_path}")
    print(f" Модель готова к инференсу!")
    print(f"\nИспользование модели:")
    print(f"""
from ultralytics import YOLO

model = YOLO('{final_model_path}')
results = model.predict('path/to/image.jpg', conf=0.5)
    """)
else:
    print(" Модель не найдена")

print("\n" + "=" * 70)
print("ОБУЧЕНИЕ ЗАВЕРШЕНО!")
print("=" * 70)
print("\n Следующий шаг:")
print("   Перейдите к ноутбуку: 03_evaluation_and_demo.ipynb")


In [ ]:
print("=" * 70)
print("ОЦЕНКА И ДЕМОНСТРАЦИЯ МОДЕЛИ (person-weapon-v2)")
print("=" * 70 + "\n")


# ЧАСТЬ 1: ЗАГРУЗКА МОДЕЛИ

In [ ]:

print("=" * 70)
print("ЗАГРУЗКА МОДЕЛИ")
print("=" * 70 + "\n")

from ultralytics import YOLO

BASE_DIR = Path('data')
MODELS_DIR = Path('models')
RESULTS_DIR = Path('results')
RESULTS_DIR.mkdir(exist_ok=True)

MODEL_PATH = MODELS_DIR / 'yolov8n_best.pt'

if MODEL_PATH.exists():
    print(f"Загрузка модели: {MODEL_PATH}")
    model = YOLO(str(MODEL_PATH))
    print(f"Модель загружена успешно\n")
else:
    print(f"ОШИБКА: Модель не найдена по пути: {MODEL_PATH}")
    print("Запустите сначала: 02_model_training.ipynb")
    MODEL_PATH = None

# Загрузить конфиг
YAML_PATH = BASE_DIR / 'dataset.yaml'
if YAML_PATH.exists():
    with open(YAML_PATH, 'r') as f:
        config = yaml.safe_load(f)
    CLASS_NAMES = config['names']
    print(f"Классы: {CLASS_NAMES}\n")
else:
    CLASS_NAMES = {0: 'grenade', 1: 'knife', 2: 'pistol', 3: 'rifle', 4: 'person'}
    print(f"Конфиг не найден, используются стандартные классы\n")

# Определения
WEAPON_CLASSES = {0, 1, 2, 3}
PERSON_CLASS = 4

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')


# ЧАСТЬ 2: ИНФЕРЕНС НА TEST СЕТЕ И ОПРЕДЕЛЕНИЕ БРАКОНЬЕРОВ


In [ ]:

print("=" * 70)
print("ИНФЕРЕНС И ОПРЕДЕЛЕНИЕ БРАКОНЬЕРОВ")
print("=" * 70 + "\n")

if MODEL_PATH and MODEL_PATH.exists():
    TEST_IMAGES_DIR = BASE_DIR / 'processed' / 'yolo_format' / 'test' / 'images'

    if TEST_IMAGES_DIR.exists():
        test_images = list(TEST_IMAGES_DIR.glob('*.*'))
        print(f"Найдено тестовых изображений: {len(test_images)}\n")

        print("Запуск инференса и определения браконьеров...\n")

        all_results = []
        poacher_detections = []
        person_only_detections = []
        weapon_only_detections = []

        for img_path in tqdm(test_images, desc='Анализ'):
            results = model.predict(str(img_path), conf=0.5, verbose=False)

            for result in results:
                boxes = result.boxes
                classes_in_image = set(int(box.cls) for box in boxes)

                # Определить тип содержимого
                has_person = PERSON_CLASS in classes_in_image
                has_weapon = any(w in classes_in_image for w in WEAPON_CLASSES)

                all_results.append({
                    'image': img_path,
                    'results': result,
                    'has_person': has_person,
                    'has_weapon': has_weapon
                })

                # Классифицировать
                if has_person and has_weapon:
                    poacher_detections.append((img_path, result))
                elif has_person:
                    person_only_detections.append((img_path, result))
                elif has_weapon:
                    weapon_only_detections.append((img_path, result))

        print(f"\nИнференс завершён для {len(test_images)} изображений")
    else:
        print(f"️Test dataset не найден: {TEST_IMAGES_DIR}")
        all_results = []
        poacher_detections = []

# ЧАСТЬ 3: СТАТИСТИКА БРАКОНЬЕРОВ


In [ ]:

print("\n" + "=" * 70)
print("СТАТИСТИКА ОПРЕДЕЛЕНИЯ БРАКОНЬЕРОВ")
print("=" * 70 + "\n")

total_images = len(all_results)
poacher_count = len(poacher_detections)
person_only_count = len(person_only_detections)
weapon_only_count = len(weapon_only_detections)

print("Результаты анализа:")
print(f"   Всего тестовых изображений: {total_images}")
print(f"   Браконьеры обнаружены: {poacher_count} ({poacher_count / total_images * 100:.1f}%)")
print(f"   Только люди: {person_only_count} ({person_only_count / total_images * 100:.1f}%)")
print(f"   Только оружие: {weapon_only_count} ({weapon_only_count / total_images * 100:.1f}%)")

# Подсчитать объекты
weapon_stats = {i: 0 for i in range(5)}
for result in all_results:
    for box in result['results'].boxes:
        class_id = int(box.cls)
        weapon_stats[class_id] += 1

print(f"\n📈 Статистика по классам:")
for class_id in range(5):
    count = weapon_stats[class_id]
    class_name = CLASS_NAMES[class_id]
    print(f"   {class_name:10s}: {count:4d} объектов")

# ЧАСТЬ 4: ВИЗУАЛИЗАЦИЯ БРАКОНЬЕРОВ


In [ ]:

print("\n" + "=" * 70)
print("ПРИМЕРЫ ДЕТЕКЦИЙ БРАКОНЬЕРОВ")
print("=" * 70 + "\n")


def draw_detections(image_path, result, class_names, weapon_classes, person_class):
    """
    Нарисовать детекции на изображении с цветовой кодировкой.
    """
    image = cv2.imread(str(image_path))
    if image is None:
        return None

    colors = {
        0: (0, 165, 255),  # grenade - оранжевый
        1: (0, 255, 255),  # knife - жёлтый
        2: (0, 0, 255),  # pistol - красный
        3: (255, 0, 0),  # rifle - синий
        4: (0, 255, 0)  # person - зелёный
    }

    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        class_id = int(box.cls)
        confidence = float(box.conf)

        color = colors.get(class_id, (255, 255, 255))

        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

        label = f"{class_names[class_id]} {confidence:.2f}"
        cv2.putText(image, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


# Показать примеры браконьеров
n_poachers = min(6, len(poacher_detections))
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx in range(n_poachers):
    img_path, result = poacher_detections[idx]
    annotated_img = draw_detections(img_path, result, CLASS_NAMES, WEAPON_CLASSES, PERSON_CLASS)

    if annotated_img is not None:
        axes[idx].imshow(annotated_img)
        axes[idx].set_title(f'БРАКОНЬЕР: {img_path.name}', fontweight='bold', color='red')
        axes[idx].axis('off')

plt.suptitle('Примеры обнаруженных браконьеров', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"Примеры браконьеров визуализированы\n")


# ЧАСТЬ 5: РАСПРЕДЕЛЕНИЕ ТИПОВ ОРУЖИЯ СРЕДИ БРАКОНЬЕРОВ

In [ ]:

print("=" * 70)
print("АНАЛИЗ ТИПОВ ОРУЖИЯ У БРАКОНЬЕРОВ")
print("=" * 70 + "\n")

weapon_types = {'grenade': 0, 'knife': 0, 'pistol': 0, 'rifle': 0}

for img_path, result in poacher_detections:
    for box in result.boxes:
        class_id = int(box.cls)
        if class_id in WEAPON_CLASSES:
            weapon_name = CLASS_NAMES[class_id]
            weapon_types[weapon_name] += 1

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# График 1: Типы оружия у браконьеров
axes[0].bar(weapon_types.keys(), weapon_types.values(),
            color=['#FF6B35', '#FFD23F', '#FF0000', '#0066FF'], alpha=0.7)
axes[0].set_ylabel('Количество', fontsize=12)
axes[0].set_title('Типы оружия у обнаруженных браконьеров', fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

for i, (k, v) in enumerate(weapon_types.items()):
    axes[0].text(i, v + 0.5, str(v), ha='center', fontweight='bold')

# График 2: Распределение тип контента
types_count = [poacher_count, person_only_count, weapon_only_count]
types_labels = ['Браконьеры', 'Только люди', 'Только оружие']
colors = ['#e74c3c', '#3498db', '#f39c12']

axes[1].bar(types_labels, types_count, color=colors, alpha=0.7)
axes[1].set_ylabel('Количество изображений', fontsize=12)
axes[1].set_title('Распределение типов содержимого', fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

for i, v in enumerate(types_count):
    axes[1].text(i, v + 1, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("Анализ завершён\n")


# ЧАСТЬ 6: СОХРАНЕНИЕ РЕЗУЛЬТАТОВ

In [ ]:

print("=" * 70)
print("СОХРАНЕНИЕ РЕЗУЛЬТАТОВ")
print("=" * 70 + "\n")

results_json = {
    "evaluation_results": {
        "total_test_images": total_images,
        "poachers_detected": poacher_count,
        "person_only": person_only_count,
        "weapon_only": weapon_only_count,
        "detection_rate": f"{poacher_count / total_images * 100:.1f}%"
    },
    "weapon_statistics": weapon_types,
    "class_detections": weapon_stats,
    "model": str(MODEL_PATH),
    "timestamp": str(pd.Timestamp.now())
}

results_path = RESULTS_DIR / 'poacher_detection_results.json'
with open(results_path, 'w') as f:
    json.dump(results_json, f, indent=2, ensure_ascii=False)

print(f"Результаты сохранены: {results_path}\n")


# ЧАСТЬ 7: ФУНКЦИЯ ДЛЯ ИСПОЛЬЗОВАНИЯ В PRODUCTION

In [ ]:

print("=" * 70)
print("ФУНКЦИЯ ОПРЕДЕЛЕНИЯ БРАКОНЬЕРОВ")
print("=" * 70 + "\n")


def detect_and_classify_poachers(image_path, model, class_names,
                                 weapon_classes={0, 1, 2, 3}, person_class=4,
                                 conf_threshold=0.5):
    """
    Обнаружить и классифицировать браконьеров на изображении.

    Args:
        image_path: путь к изображению
        model: обученная модель YOLOv8
        class_names: словарь названий классов
        weapon_classes: набор ID классов оружия
        person_class: ID класса человека
        conf_threshold: порог уверенности

    Returns:
        {
            'is_poacher': bool,
            'detections': [{'class': str, 'confidence': float, 'bbox': list}],
            'weapons_found': list,
            'alert_level': str  # 'SAFE', 'PERSON', 'WEAPON', 'POACHER_ALERT'
        }
    """
    results = model.predict(str(image_path), conf=conf_threshold, verbose=False)

    detections = []
    weapons_found = []
    classes_in_image = set()

    for result in results:
        for box in result.boxes:
            class_id = int(box.cls)
            classes_in_image.add(class_id)

            detection = {
                'class': class_names[class_id],
                'confidence': float(box.conf),
                'bbox': [int(x) for x in box.xyxy[0].tolist()],
            }
            detections.append(detection)

            if class_id in weapon_classes:
                weapons_found.append(class_names[class_id])

    # Определить уровень угрозы
    has_person = person_class in classes_in_image
    has_weapon = any(w in classes_in_image for w in weapon_classes)

    if has_person and has_weapon:
        alert_level = 'POACHER_ALERT'
        is_poacher = True
    elif has_person:
        alert_level = 'PERSON'
        is_poacher = False
    elif has_weapon:
        alert_level = 'WEAPON'
        is_poacher = False
    else:
        alert_level = 'SAFE'
        is_poacher = False

    return {
        'is_poacher': is_poacher,
        'detections': detections,
        'weapons_found': list(set(weapons_found)),
        'alert_level': alert_level
    }


print(" Функция detect_and_classify_poachers() готова к использованию!\n")

print("Пример использования:")
print("""
# Анализировать одно изображение
result = detect_and_classify_poachers(
    'path/to/image.jpg',
    model,
    CLASS_NAMES,
    weapon_classes={0,1,2,3},
    person_class=4
)

if result['is_poacher']:
    print(f"ВНИМАНИЕ: Браконьер обнаружен!")
    print(f"Типы оружия: {result['weapons_found']}")
else:
    print(f"Уровень угрозы: {result['alert_level']}")

# Батч-обработка директории
from pathlib import Path

for img_file in Path('images/').glob('*.jpg'):
    result = detect_and_classify_poachers(str(img_file), model, CLASS_NAMES)
    if result['is_poacher']:
        print(f"{img_file.name} - БРАКОНЬЕР!")
""")

In [ ]:
# ============================================================================
# ЧАСТЬ 8: ИТОГОВЫЙ ОТЧЁТ
# ============================================================================

print("\n" + "=" * 70)
print("ИТОГОВЫЙ ОТЧЁТ")
print("=" * 70 + "\n")

print("Модель успешно обучена и оценена!\n")

print(f"Итоги:")
print(f"   Модель: YOLOv8 Nano (person-weapon-v2)")
print(f"   Классы: {list(CLASS_NAMES.values())}")
print(f"   Test изображений: {total_images}")
print(f"   Браконьеры обнаружены: {poacher_count}")
print(f"   Точность обнаружения: {poacher_count / total_images * 100:.1f}%")

print(f"\nФайлы:")
print(f"   Модель: {MODEL_PATH}")
print(f"   Результаты: {results_path}")

print(f"\nЛогика определения браконьера:")
print(f"   IF (Person обнаружен) AND (Оружие обнаружено)")
print(f"   THEN -> БРАКОНЬЕР (Alert Level: POACHER_ALERT)")

print(f"\nДЕМОНСТРАЦИЯ ЗАВЕРШЕНА!\n")

print("Модель готова к использованию в production!")
